In [1]:
!pip install --upgrade --quiet opik langchain langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.5/185.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/3

In [4]:
import opik

opik.configure(use_local=False)

OPIK: Your Opik cloud API key is available at https://www.comet.com/api/my/settings/.


Please enter your Opik Cloud API key:··········
Do you want to use "sumitmishra5504" workspace? (Y/n)Y


OPIK: Saved configuration to a file: /root/.opik.config


In [5]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


## Prompt 1

In [22]:
from opik import Opik, track, DatasetItem
from opik.evaluation import evaluate
from opik.evaluation.metrics import Equals, Hallucination
from opik.integrations.openai import track_openai
import openai

# Define the task to evaluate
openai_client = track_openai(openai.OpenAI())
os.environ["OPIK_PROJECT_NAME"] = "opik-prompt-refinement"

MODEL = "gpt-4o-mini"

from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name=MODEL,
    max_tokens=200
)

@track(name="prompt_1_tracing")
def get_sentiment(messages):
    return llm.invoke(messages).content

def build_input_prompt(statement):
    user_content = f"""You will be provided a statement from twitter and your job is to determine sentiments of it. Sentiments should be below.
    sentiments: ["sadness", "happiness", "neutral"]

    Here is the statement.
    <statement>{statement}</statement>
    """
    messages = [{'role': 'user', 'content': user_content}]
    return messages

# Define the evaluation task
def evaluation_task(x: DatasetItem):
    return {
        "input": x.input['input'],
        "output": get_sentiment(build_input_prompt(x.input['input'])),
        "expected_output": x.expected_output['golden_answer']
    }

# Create a simple dataset
client = Opik()
dataset = client.get_dataset(name="Sentiment Dataset")

# Define the metrics
from opik.evaluation.metrics import ContextPrecision, ContextRecall

metric1 = ContextPrecision()
metric2 = ContextRecall()

evaluation = evaluate(
    experiment_name="Prompt 1 Exp",
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=[metric1, metric2],
    experiment_config={
        "model": MODEL
    }
)

Evaluation: 100%|██████████| 20/20 [00:05<00:00,  3.65it/s]


╭─ Sentiment Dataset (20 samples) ──────────────────╮
│                                                   │
│ Total time:        00:00:05                       │
│ Number of samples: 20                             │
│                                                   │
│ context_precision_metric: 0.6105 (avg) - 1 failed │
│ context_recall_metric: 0.7100 (avg)               │
│                                                   │
╰───────────────────────────────────────────────────╯

Uploading results to Opik ...

## Prompt 2

In [23]:
from opik import Opik, track, DatasetItem
from opik.evaluation import evaluate
from opik.evaluation.metrics import Equals, Hallucination
from opik.integrations.openai import track_openai
import openai

# Define the task to evaluate
openai_client = track_openai(openai.OpenAI())
os.environ["OPIK_PROJECT_NAME"] = "opik-prompt-refinement"

MODEL = "gpt-4o-mini"

from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name=MODEL,
    max_tokens=200
)

@track(name="prompt_2_tracing")
def get_sentiment(messages):
    return llm.invoke(messages).content

def build_input_prompt(statement):
    user_content = f"""You will be provided a statement from twitter and your job is to determine sentiments of it. Sentiments should be below and just give sentiment as response.
    sentiments: ["sadness", "happiness", "neutral"]

    Here is the statement.
    <statement>{statement}</statement>

    What is the sentiment of statement? Please respond with a only one word. e.g. sadness"""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

# Define the evaluation task
def evaluation_task(x: DatasetItem):
    return {
        "input": x.input['input'],
        "output": get_sentiment(build_input_prompt(x.input['input'])),
        "expected_output": x.expected_output['golden_answer']
    }

# Create a simple dataset
client = Opik()
dataset = client.get_dataset(name="Sentiment Dataset")

# Define the metrics
from opik.evaluation.metrics import ContextPrecision, ContextRecall

metric1 = ContextPrecision()
metric2 = ContextRecall()

evaluation = evaluate(
    experiment_name="Prompt 2 Exp",
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=[metric1, metric2],
    experiment_config={
        "model": MODEL
    }
)

Evaluation: 100%|██████████| 20/20 [00:06<00:00,  3.03it/s]


╭─ Sentiment Dataset (20 samples) ──────────────────╮
│                                                   │
│ Total time:        00:00:07                       │
│ Number of samples: 20                             │
│                                                   │
│ context_precision_metric: 0.4211 (avg) - 1 failed │
│ context_recall_metric: 0.6900 (avg)               │
│                                                   │
╰───────────────────────────────────────────────────╯

Uploading results to Opik ...